In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
import xgboost as xg 
from mlxtend.regressor import StackingCVRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df1=pd.read_csv("dataset.csv")

In [ ]:
df1.head()

,segment_id,sensor_1_sum,sensor_1_mean,sensor_1_max,sensor_1_min,sensor_1_std,sensor_1_diff_min_max,sensor_1_diff_mean_max,sensor_1_quantile25,sensor_1_quantile50,sensor_1_quantile75,sensor_1_quantile99,sensor_1_mean_10000,sensor_1_mean_10000l,sensor_1_std_10000,sensor_1_std_10000l,sensor_2_sum,sensor_2_mean,sensor_2_max,sensor_2_min,sensor_2_std,sensor_2_diff_min_max,sensor_2_diff_mean_max,sensor_2_quantile25,sensor_2_quantile50,sensor_2_quantile75,sensor_2_quantile99,sensor_2_mean_10000,sensor_2_mean_10000l,sensor_2_std_10000,sensor_2_std_10000l,sensor_3_sum,sensor_3_mean,sensor_3_max,sensor_3_min,sensor_3_std,sensor_3_diff_min_max,sensor_3_diff_mean_max,sensor_3_quantile25,sensor_3_quantile50,...,sensor_1_rolling_std_100,sensor_1_rolling_max_100,sensor_1_rolling_min_100,sensor_2_rolling_mean_100,sensor_2_rolling_std_100,sensor_2_rolling_max_100,sensor_2_rolling_min_100,sensor_3_rolling_mean_100,sensor_3_rolling_std_100,sensor_3_rolling_max_100,sensor_3_rolling_min_100,sensor_4_rolling_mean_100,sensor_4_rolling_std_100,sensor_4_rolling_max_100,sensor_4_rolling_min_100,sensor_5_rolling_mean_100,sensor_5_rolling_std_100,sensor_5_rolling_max_100,sensor_5_rolling_min_100,sensor_6_rolling_mean_100,sensor_6_rolling_std_100,sensor_6_rolling_max_100,sensor_6_rolling_min_100,sensor_7_rolling_mean_100,sensor_7_rolling_std_100,sensor_7_rolling_max_100,sensor_7_rolling_min_100,sensor_8_rolling_mean_100,sensor_8_rolling_std_100,sensor_8_rolling_max_100,sensor_8_rolling_min_100,sensor_9_rolling_mean_100,sensor_9_rolling_std_100,sensor_9_rolling_max_100,sensor_9_rolling_min_100,sensor_10_rolling_mean_100,sensor_10_rolling_std_100,sensor_10_rolling_max_100,sensor_10_rolling_min_100,time_to_eruption
0,698305523,200457.0,3.340894,2931.0,-3019.0,367.633890,-88.0,2927.659106,-214.0,0.0,219.0,970.0,4.9501,1.0707,432.279698,284.327545,139207.0,2.320078,4738.0,-6015.0,514.740948,-1277.0,4735.679922,-245.0,0.0,250.0,1456.0,9.9759,-31.2292,607.668508,376.987984,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,12560.427735,101000.416381,-100518.814577,109.801028,7318.858618,82460.650587,-65971.628929,0.000000,0.000000,0.000000,0.000000,-101.702544,9658.821046,37860.546906,-41859.374593,-18.111054,2198.300876,25286.849071,-30101.034547,770.053448,52707.620672,165219.792330,-145440.692262,-413.937449,18213.129008,83060.769122,-68436.016554,90.522089,26553.606751,90926.196470,-90599.871335,0.000000,0.000000,0.000000,0.000000,65.655065,15043.202930,114344.945877,-115229.604770,13903256
1,1191644191,-165217.0,-2.753571,4223.0,-4154.0,611.311853,69.0,4220.246429,-376.0,0.0,369.0,1557.0,-3.4140,-1.6858,766.909079,530.485286,-8315.0,-0.138581,20325.0,-11098.0,1140.506282,9227.0,20324.861419,-582.0,0.0,582.0,3057.0,-1.1737,-3.3488,1485.371941,1047.332342,-3407912.0,-56.797587,4863.0,-4617.0,1213.212902,246.0,4806.202413,-776.0,0.0,...,17872.470064,63185.450933,-75414.603451,-7.568476,13309.511555,162725.319550,-213930.268345,-2787.964789,56027.833712,190487.754195,-204473.438197,-66.206437,16822.822031,61979.902191,-61442.031437,-2.798846,473.047632,9352.818705,-7071.722522,751.287779,41078.410030,135828.790562,-126549.163249,372.027733,31667.498627,128111.273316,-97630.384883,206.703481,37717.767319,111428.176279,-150875.433831,52.105823,12029.097913,70767.791070,-71823.590341,499.870725,30508.686691,126078.853724,-164003.213363,5525566
2,1403440092,-95822.0,-1.597007,2118.0,-1918.0,306.984282,200.0,2116.402993,-184.0,0.0,186.0,748.0,-2.0455,-4.0018,288.242237,269.900865,-10101.0,-0.168347,8604.0,-6475.0,514.620059,2129.0,8603.831653,-254.0,0.0,249.0,1369.0,-1.2282,-0.0584,454.868550,333.709113,-192776.0,-3.212880,1335.0,-1646.0,259.603885,-311.0,1331.787120,-165.0,0.0,...,8400.388923,70925.556339,-57435.023158,-13.389881,5652.716551,57149.347197,-38962.240673,-150.096677,7317.215505,36723.636479,-39184.035652,19.749261,6584.465991,37627.717487,-33677.438245,-44.171727,2082.616831,26549.361167,-28961.676900,96.706874,23969.962904,72166.436490,-80872.28

In [ ]:
Y=df1["time_to_eruption"]
X=df1.drop(columns=['time_to_eruption',"segment_id"])

### Splitting the dataset

In [ ]:
Y

0       13903256
1        5525566
2        4524342
3       19863157
4       14951435
          ...   
4426    36335121
4427     3819695
4428    32331124
4429    24993068
4430     5288221
Name: time_to_eruption, Length: 4431, dtype: int64

In [ ]:
#80-20 split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Normalising the Dataset

In [ ]:
# Since the range of values across different columns are different we are Normalizing the Dataset 
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# https://www.geeksforgeeks.org/standardscaler-minmaxscaler-and-robustscaler-techniques-ml/
scaler=MinMaxScaler()
scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(scaled, columns =X_train.columns.values )
scaled_test=scaler.transform(X_test)
X_test = pd.DataFrame(scaled_test, columns =X_test.columns.values )

In [ ]:
#50-50 split
D1,D2,Y1,Y2 = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [ ]:
print(D1.shape)
print(D2.shape)
print(X_test.shape)

(1772, 1100)
(1772, 1100)
(887, 1100)


### The Base And Meta Models

In [ ]:
# initialising the models
lass= Lasso(tol=0.1)
rdge = Ridge(tol=0.1)
svr1 = SVR(kernel='poly')
svr2 = SVR(kernel='rbf')
svr3 = SVR(kernel='linear')
dcsn = DecisionTreeRegressor()
lgbm = lgb.LGBMRegressor(boosting_type ='gbdt')

rf = RandomForestRegressor()

### Custom Model

In [ ]:
#Selecting random rows fromthe dataset
#https://www.geeksforgeeks.org/how-to-randomly-select-rows-from-pandas-dataframe/
def random_rows(X,Y):
  """Function to sample 70%  Data with Replacements"""
  chosen_idx = np.random.choice(len(X), replace = True, size =int(0.7*len(X))) 
  X1=X.iloc[chosen_idx]
  Y1=Y.iloc[chosen_idx]
  return X1,Y1

In [ ]:
#Creating n samples from selected rows
def nsamples (X,Y,n):
  """Function to create the list on n samples"""
  selected_in_row=[]
  selected_out_row=[]
  for i in range(0,n):
    a,b = random_rows(X,Y)
    selected_in_row.append(a)
    selected_out_row.append(b)
  return selected_in_row,selected_out_row

In [ ]:
# Base Model
def base_model (X,Y,n):
  """Function to train the k samples with base model and stores them in a list """
# list of base models with tuned parameters
  models=[Lasso(tol=0.1), Ridge(tol=0.1),SVR(kernel='poly'),SVR( kernel='rbf'),SVR(kernel='linear'),DecisionTreeRegressor(max_depth=7),lgb.LGBMRegressor(boosting_type ='gbdt',max_depth=5) ]
  model=[0]*n
# Training the base model for n samples
  for i in range(0,n):
    model[i]= models[i]
    model[i].fit(X[i],Y[i])
  return model

In [ ]:
# Function to predict and create new dataset
def k_pred(base,X):
  """Function to predict the K features using K Base Model and to store them as a DataFrame"""
  y_pred=[0]*len(base)
  df=pd.DataFrame()
# Predicting the values for n samples
  for i in range(0,len(base)):
    y_pred[i]=base[i].predict(X)
# Saving the resut in dataframe
    df["prediction_{}".format(i+1)]=y_pred[i]  
  return df

In [ ]:
# Function to predict the final Result
def evaluation(base,meta,X):
  """Function to predict K Features using Base models and to use that features to predict final result using a meta model"""
  y_pred=[0]*len(base)
  df1=pd.DataFrame()
# Predicting the base model
  for i in range(0,len(base)):
    y_pred[i]=base[i].predict(X)
# Save base model as features
    df1["prediction_{}".format(i+1)]=y_pred[i] 
# predicting the meta model
  yhat=meta.predict(df1)
  return yhat

In [ ]:
# Custom Ensemble
def custom_en(n):
  """Custom Ensemble including Both base and Meta Models"""
  in_sample,tar_sample= nsamples(D1,Y1,n)

  # Training  using the Base Model
  base=base_model(in_sample,tar_sample,n)
  df=k_pred(base,D2)

  # Random Forest as Meta Model
  # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
  gbdt = RandomForestRegressor(min_samples_split=5)
  meta=gbdt.fit(df,Y2)

  # storing the final result
  y_hat=evaluation(base,meta,X_test)
  return y_hat

In [ ]:
# Finding the best parameters
params = [3,5,7]
# Saving the scores for each parameter
scores=[]
for i in range(0,len(params)):
    y_hat = custom_en(params[i])
    scores.append ((params[i],mean_absolute_error( y_test , y_hat )))

In [ ]:
scores

[(3, 6616899.923314216), (5, 6603642.577418238), (7, 3653445.2630618704)]

In [ ]:
# finding the paameter for minimum score
# https://stackoverflow.com/questions/3282823/get-the-key-corresponding-to-the-minimum-value-within-a-dictionary
min(scores, key=lambda x: x[1]) 

(7, 3653445.2630618704)

In [ ]:
y_hat = custom_en(7)

In [ ]:
score=mean_absolute_error( y_test , y_hat )
score

5292222.674077656